In [2]:
import nltk
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from  sklearn.decomposition import NMF
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patheffects as PathEffects
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from sklearn.preprocessing import normalize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yefeng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv('data/OnTheSnow_SkiAreaReviews.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18262 entries, 0 to 18261
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   State                          18262 non-null  object
 1   Ski Area                       18262 non-null  object
 2   Reviewer Name                  18153 non-null  object
 3   Review Date                    18262 non-null  object
 4   Review Star Rating (out of 5)  18262 non-null  int64 
 5   Review Text                    18250 non-null  object
dtypes: int64(1), object(5)
memory usage: 856.2+ KB


In [11]:
len(df['Ski Area'].unique())

291

In [10]:
df['sentence'] = [sent_tokenize(str(rev)) for rev in df['Review Text']]

In [20]:
df['sentence']

0        [I'm glad our family experienced Squaw but I w...
1        [I went skiing today 5/22, granite chief was a...
2        [We had a horrible experience on our family sk...
3        [This is the first year I ski Squaw., I've bee...
4        [Both Squaw and Alpine have incredible terrain...
                               ...                        
18257    [Looks like they only made snow on one trail.,...
18258    [Looks cute from highway........ but not enoug...
18259    [Place has incredible potential., Could be bes...
18260    [Great exposure from interstate......... but l...
18261    [My dad and I visited on a whim on our way bac...
Name: sentence, Length: 18262, dtype: object

In [11]:
df['rev_id'] = [i for i in range(len(df))]

In [12]:
# create a daataframe with a sentence per row


rev_list = df['sentence'].tolist()
rev_id = df['rev_id'].tolist()

# create a list of all sentences and all review IDs of the same length
sentence_list = []
review_id = []
for i, rec in enumerate(rev_list):
    for sent in rec:
        sentence_list.append(sent)
        review_id.append(rev_id[i])

In [13]:
sent_dict = {}
sent_dict['rev_id'] = review_id
sent_dict['sentence'] = sentence_list

df_sent = pd.DataFrame.from_dict(sent_dict)

In [15]:
def lemmatizer(text):
    tokens = word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    lemmatizer = WordNetLemmatizer()
    lem_words = []
    for tagged_word in tagged:
        word = tagged_word[0].lower()
        if tagged_word[1].startswith('VB'):
            word_tag = 'v'
        elif tagged_word[1].startswith('JJ'):
            word_tag = 'a'
        elif tagged_word[1].startswith('RB'):
            word_tag = 'r'
        else:
            word_tag = 'n'
                 
        lem_words.append(lemmatizer.lemmatize(word,pos=word_tag))
    return(lem_words)

In [16]:
df_sent['lem_sent_token'] = [lemmatizer(str(rev)) for rev in df_sent['sentence']]

In [17]:
df_sent['lem_sent_text'] = [" ".join(lems) for lems in df_sent['lem_sent_token']]

In [18]:
df_sent.head()

,rev_id,sentence,lem_sent_token,lem_sent_text
0,0,I'm glad our family experienced Squaw but I wo...,"[i, 'm, glad, our, family, experience, squaw, ...",i 'm glad our family experience squaw but i wo...
1,0,Here is a list of cons in detail: \n1.,"[here, be, a, list, of, con, in, detail, :, 1, .]",here be a list of con in detail : 1 .
2,0,The staff were moody which is odd for an expen...,"[the, staff, be, moody, which, be, odd, for, a...",the staff be moody which be odd for an expensi...
3,0,2.,"[2, .]",2 .
4,0,The on hill ski staff had the worst ego's.,"[the, on, hill, ski, staff, have, the, bad, eg...",the on hill ski staff have the bad ego 's .


In [21]:
with open('data/filtered_sent_lemm.pkl', 'wb') as picklefile:
    pickle.dump(df_sent, picklefile)